In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import pandas as pd
from astroquery.simbad import Simbad
from tqdm import tqdm
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
from astroquery.sdss import SDSS
import astropy.units as u
import astropy
import glob
import os

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [2]:
table = astropy.table.Table.read('./data/tables/old_xmatch.fits')

In [3]:
try:
    os.mkdir('./data/spectra')
except:
    print('spectra folder already exists!')

spectra folder already exists!


In [4]:
allspec = glob.glob('./data/spectra/*')

table['plate'] = np.zeros(len(table)).astype(int)
table['mjd'] = np.zeros(len(table)).astype(int)
table['fiber'] = np.zeros(len(table)).astype(int)


for obj in table:
    arr = obj['pmf'].split('-')
    obj['plate'] = arr[0]
    obj['mjd'] = arr[1]
    obj['fiber'] = arr[2]

In [5]:
table

dist,solution_id,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,astrometric_weight_al,astrometric_pseudo_colour,astrometric_pseudo_colour_error,mean_varpi_factor_al,astrometric_matched_observations,visibility_periods_used,astrometric_sigma5d_max,frame_rotator_object_type,matched_observations,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_nb_transits,rv_template_teff,rv_template_logg,rv_template_fe_h,l,b,ecl_lon,ecl_lat,priam_flags,teff_val,teff_percentile_lower,teff_percentile_upper,a_g_val,a_g_percentile_lower,a_g_percentile_upper,e_bp_min_rp_val,e_bp_min_rp_percentile_lower,e_bp_min_rp_percentile_upper,flame_flags,radius_val,radius_percentile_lower,radius_percentile_upper,lum_val,lum_percentile_lower,lum_percentile_upper,keplertable_oid,pmf,sng,u,su,g,sg,r,sr,i,si,z,sz,ebv,ppm,l_2,b_2,ty,ra_2,dec_2,plate,mjd,fiber
,,,,yr,deg,mas,deg,mas,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,mas,,,,1 / mas2,1 / um,1 / um,,,,mas,,,,,'electron'.s**-1,'electron'.s**-1,,mag,,'electron'.s**-1,'electron'.s**-1,,mag,,'electron'.s**-1,'electron'.s**-1,,mag,,,mag,mag,mag,km / s,km / s,,K,log(cm.s**-2),'dex',deg,deg,deg,deg,,K,K,K,mag,mag,mag,mag,mag,mag,,solRad,solRad,solRad,solLum,solLum,solLum,,,,,,,,,,,,,,,,,,,,,,,
float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,int32,float32,float32,float64,float64,int16,bool,float32,float64,float64,float32,int16,int16,float32,int32,int16,bool,int32,float64,float64,float32,float32,int32,float64,float64,float32,float32,int32,float64,float64,float32,float32,float32,int16,float32,float32,float32,float64,float64,int32,float32,float32,float32,float64,float64,float64,float64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,float32,float32,float32,float32,float32,float32,int32,bytes16,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes29,float64,float64,int64,int64,int64
6.886723761375954e-05,1635721458409799680,3840044772324604800,246455343,2015.5,145.2811144339209,0.26144973949694056,-0.33040417034419794,0.2518442668684157,3.880680898040339,0.31003344324857224,12.516975,-2.177196884035969,0.5001781292018255,17.667364788022233,0.44290444889844005,-0.15225786,-0.32449538,-0.15401109,0.27266255,-0.12729257,0.13943188,-0.22490928,0.21420127,-0.31991842,-0.070016004,124,0,124,0,0.927769,133.17453,0.538483766979828,0.7443576533952183,31,False,0.29010165,1.8315765626687603,0.07931570256092887,-0.14118858,14,9,0.4597729,0,15,False,132,464.1249560102739,1.8297966999979707,253.64838,19.021778,14,312.1753609645827,4.761797022602632,65.55831,19.11539,13,205.50138680892428,11.809151370651627,17.401876,18.979883,1.1153823,0,0.13550758,0.09361267,0.041894913,nan,nan,0,nan,nan,nan,235.8913153464539,36.70747519159311,147.66818726026315,-13.405482406987748,--,nan,nan,nan,nan,nan,nan,nan,nan,nan,--,nan,nan,nan,nan,nan,nan,1,0266-51602-0314,12,19.369,0.023,18.965,0.013999999999999999,19.063,0.012,19.23099

For John: you can use only the magnetic stars to start with, if you want. You can select out the 'DAH' stars:

In [6]:
puremagnetic = (
    table['ty'] == 'DAH'
)

In [7]:
len(table[puremagnetic])

351

Let's download spectra!

In [8]:
count = 1

sel1 = 0
sel2 = len(table) # you can use these variables to select 'chunks' of the table to download in batches

spec_path = './data/spectra/'
for jj in tqdm(np.arange(sel1,sel2)):
    obj = table[jj]
    objid = obj['source_id']
    
    filename = './data/spectra/' + str(objid) + '_spec_0.fits'
    if filename in allspec:
        continue; # if the spectrum has already been downloaded and is in the folder, we skip it. 

    try:
        xid = SDSS.query_specobj(plate = obj['plate'], mjd = obj['mjd'], fiberID = obj['fiber'])
        
        # We lookup SDSS spectra using the plate, mjd, and fiber identifier ^
        
    except ValueError:
        print('unknown error')

    if xid == None:
        print('object not found in SDSS... skipping...')
        continue

    sp = SDSS.get_spectra(matches=xid)
    for i in range(15680, len(sp)): # we select 'all' spectra for this object - so multiple spectra will be downloaded 
                             # for objects with multiple visits, and given _spec_0, spec_1 etc. 
        try:
            sp[i].writeto(spec_path + str(objid) + '_spec_' + str(i) + '.fits') # write the spectra to file
            if i % 100 == 0:
                print('spectrum downloaded!')
        except OSError:
            print("File already exists!")
    count += 1 

 45%|████▍     | 15680/35080 [12:20:08<15:15:43,  2.83s/it]spectrum downloaded!



ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))